<a href="https://colab.research.google.com/github/clashleyuncc/Intro-To-ML/blob/main/HW6P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#use gpu?

import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
import time

start_time = time.time()

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            print(f"Training Time: {time.time()-start_time} seconds")

from google.colab import drive
#drive.mount('/content/drive')
file_path = '/content/drive/My Drive/4th Year Charlotte/IntroToML/Housing.csv'

housing = pd.read_csv(file_path)

varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

housing[varlist] = housing[varlist].apply(binary_map)

Y = housing.pop('price')
X = housing[['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning']]

X = StandardScaler().fit_transform(X)

Ytarget = pd.DataFrame(Y)

#print(X.mean(axis=0), X.std(axis=0))

t_c = Y
t_u = X
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

t_u = t_u.to(torch.float32)
t_c = t_c.to(torch.float32)

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]
t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]
t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

seq_model = nn.Sequential(OrderedDict([
            ('hidden_linear', nn.Linear(10, 8)),
            ('hidden_activation', nn.Tanh()),
            ('output_linear', nn.Linear(8,1))
]))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-5)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val)



Epoch 1, Training loss 25622359310336.0000, Validation loss 28579731603456.0000
Training Time: 0.03862428665161133 seconds
Epoch 1000, Training loss 18868292550656.0000, Validation loss 21541754503168.0000
Training Time: 2.0497889518737793 seconds
Epoch 2000, Training loss 14145338474496.0000, Validation loss 16581155356672.0000
Training Time: 4.197384834289551 seconds
Epoch 3000, Training loss 10850344632320.0000, Validation loss 13087668174848.0000
Training Time: 6.109996795654297 seconds
Epoch 4000, Training loss 8551580827648.0000, Validation loss 10623111921664.0000
Training Time: 7.588721513748169 seconds
Epoch 5000, Training loss 6947841507328.0000, Validation loss 8880892411904.0000
Training Time: 9.077118158340454 seconds
